
# 1/ Ingesting and preparing PDF for LLM and Self Managed Vector Search Embeddings

## In this example, we will focus on ingesting pdf documents as source for our retrieval process. 

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-pdf-self-managed-0.png?raw=true" style="float: right; width: 600px; margin-left: 10px">


For this example, we will add Databricks ebook PDFs from [Databricks resources page](https://www.databricks.com/resources) to our knowledge database.

**Note: This demo is advanced content, we strongly recommend going over the simple version first to learn the basics.**

Here are all the detailed steps:

- Use autoloader to load the binary PDFs into our first table. 
- Use the `unstructured` library  to parse the text content of the PDFs.
- Use `llama_index` or `Langchain` to split the texts into chuncks.
- Compute embeddings for the chunks.
- Save our text chunks + embeddings in a Delta Lake table, ready for Vector Search indexing.


Lakehouse AI not only provides state of the art solutions to accelerate your AI and LLM projects, but also to accelerate data ingestion and preparation at scale, including unstructured data like PDFs.

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=429769148865153&notebook=%2F03-advanced-app%2F01-PDF-Advanced-Data-Preparation&demo_name=llm-rag-chatbot&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fllm-rag-chatbot%2F03-advanced-app%2F01-PDF-Advanced-Data-Preparation&version=1">

In [0]:
%run "./0. Init"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lakehouse 0.20.0 requires packaging==24.1, but you have packaging 23.2 which is incompatible.
lakehouse 0.20.0 requires requests==2.32.3, but you have requests 2.31.0 which is incompatible.
lakehouse 0.20.0 requires urllib3==2.2.2, but you have urllib3 2.0.7 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.2.5 requires packaging<24.0,>=23.2, but you have packaging 24.1 which is incompatible.
pdfservices-sdk 4.0.0 requires requests~=2.31.0, but you have requests 2.32.3 which is incompatible.
lakehouse 0.20.0 requires certifi==2024.

/local_disk0/.ephemeral_nfs/envs/pythonEnv-344a98d7-cede-463d-94d0-b6be76673c7c/lib/python3.11/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


DataFrame[]

In [0]:
import warnings, requests, os, json
from pypdf import PdfReader
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser, MarkdownElementNodeParser
from llama_index.core import Document, set_global_tokenizer
from transformers import AutoTokenizer
from typing import Iterator
import pandas as pd
import pyspark.sql.functions as F
from mlflow.deployments import get_deploy_client

# Reduce the arrow batch size as our PDF can be big in memory
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 10)

First, we will download the PDFs from the given URL's and write them in a local folder.

Then, we will make use of **Databricks Autoloader** to automatically ingest new ones.

In [0]:
os.makedirs(PDF_ROOT_PATH, exist_ok=True)
for url in URLS:
    response = requests.get(url)
    pdf_name = url.split('/')[-1]
    if pdf_name in os.listdir(PDF_ROOT_PATH):
        continue
    with open(os.path.join(PDF_ROOT_PATH, pdf_name), 'wb') as f:
        f.write(response.content)

INFO:py4j.clientserver:Received command c on object id p0


In [0]:
%sql
CREATE VOLUME IF NOT EXISTS volume_hackathon;

In [0]:
def download_pdf(dest_path, urls):
    def download_file(url, destination):
      local_filename = url.split('/')[-1]
      with requests.get(url, stream=True) as r:
          r.raise_for_status()
          print('saving '+destination+'/'+local_filename)
          with open(destination+'/'+local_filename, 'wb') as f:
              for chunk in r.iter_content(chunk_size=8192): 
                  f.write(chunk)
      return local_filename

    if not os.path.exists(dest_path):
      os.makedirs(dest_path)
    from concurrent.futures import ThreadPoolExecutor
    def download_to_dest(url):
        download_file(url, dest_path)
    with ThreadPoolExecutor(max_workers=10) as executor:
        collections.deque(executor.map(download_to_dest, urls))

def upload_pdfs_to_volume(volume_path, urls):
  try:
    download_pdf(volume_path, urls)
  except Exception as e:
    print(str(e))

In [0]:
volume_folder =  f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/volume_hackathon".replace('`', '')
path_to_pdfs = volume_folder+"/pdfs"
upload_pdfs_to_volume(path_to_pdfs, urls=URLS)

saving /Volumes/test-catalog/bronze/volume_hackathon/pdfs/Anhaengerachsen-5_5-t-Trailer-axles-5.5-tonnes-Essieux-jus-5_5-35031401def.pdfsaving /Volumes/test-catalog/bronze/volume_hackathon/pdfs/Code_number_designations_BRO_BPW_en_2020_39342001.pdf
saving /Volumes/test-catalog/bronze/volume_hackathon/pdfs/Brake_Drum-HKN-Workshop_manual-BPW-2024-_35192401e.pdf
saving /Volumes/test-catalog/bronze/volume_hackathon/pdfs/ECO_Disc_Disc_brake_TS2_TSB_Original_spare_parts_BPW_2024_en__31232401e.pdf
saving /Volumes/test-catalog/bronze/volume_hackathon/pdfs/HKN_trailer_axles_Spare_parts_list_BPW_31022101e_2021.pdf
saving /Volumes/test-catalog/bronze/volume_hackathon/pdfs/Trailer_Disc_brake_TSB-ECODisc-_Workshop_manual_2023_en-BPW-35292301e.pdf
saving /Volumes/test-catalog/bronze/volume_hackathon/pdfs/BPW-2023-service_manual_WH-TS2_35472302e.pdf
saving /Volumes/test-catalog/bronze/volume_hackathon/pdfs/Maintenance_instructions_BPW_Trailer_Axles_and_Suspensions_2024__33112401e.pdf

404 Client Error

In [0]:
df = (
    spark.readStream
    .format('cloudFiles')
    .option('cloudFiles.format', 'BINARYFILE')
    .option("pathGlobFilter", "*.pdf")
    .load('dbfs:'+path_to_pdfs)
)

# Write the data as a Delta table
(df.writeStream
  .trigger(availableNow=True)
  .option("checkpointLocation", f'dbfs:{volume_folder}/checkpoints/raw_docs')
  .table('pdf_raw').awaitTermination())

This looks great. We'll now wrap it with a text_splitter to avoid having too big pages, and create a Pandas UDF function to easily scale that across multiple nodes.

*Note that our pdf text isn't clean. To make it nicer, we could use a few extra LLM-based pre-processing steps, asking to remove unrelevant content like the list of chapters and to only keep the core text.*

In [0]:
@F.pandas_udf("array<float>")
def get_embedding(contents: pd.Series) -> pd.Series:
    import mlflow.deployments
    deploy_client = mlflow.deployments.get_deploy_client("databricks")
    def get_embeddings(batch):
        #Note: this will fail if an exception is thrown during embedding creation (add try/except if needed) 
        response = deploy_client.predict(endpoint="databricks-gte-large-en", inputs={"input": batch})
        return [e['embedding'] for e in response.data]

    # Splitting the contents into batches of 150 items each, since the embedding model takes at most 150 inputs per request.
    max_batch_size = 150
    batches = [contents.iloc[i:i + max_batch_size] for i in range(0, len(contents), max_batch_size)]

    # Process each batch and collect the results
    all_embeddings = []
    for batch in batches:
        all_embeddings += get_embeddings(batch.tolist())

    return pd.Series(all_embeddings)

@F.pandas_udf("array<string>")
def split_in_chunks(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    def extract_and_split(content):
      txt = read_pdf(content)
      if txt is None:
        return []
      nodes = PARSER.get_nodes_from_documents([Document(text=txt)])
      return [n.text for n in nodes]

    def extract_and_split_page(txt):
      if txt is None:
        return []
      nodes = PARSER.get_nodes_from_documents([Document(text=txt)])
      return [n.text for n in nodes]
    
    for x in batch_iter:
        yield x.apply(extract_and_split_page)
        # yield x.apply(extract_and_split)

@F.pandas_udf("array<struct<title:string, page_number:int, text:string>>")
def extract_pages_content(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]: 
    for x in batch_iter:
        yield x.apply(read_pdf)

### Computing the chunk embeddings and saving them to our Delta Table

The last step is to now compute an embedding for all our documentation chunks. We already created an udf to compute the embeddings using the foundation model endpoint.

*Note that this part would typically be setup as a production-grade job, running as soon as a new documentation page is updated. <br/> This could be setup as a Delta Live Table pipeline to incrementally consume updates.*

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS hackathon_pdfs_test (
    id BIGINT GENERATED BY DEFAULT AS IDENTITY,
    path STRING,
    title STRING,
    url STRING,
    page_number INT,
    content STRING,
    embedding ARRAY<FLOAT>
) USING DELTA
TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

DataFrame[]

In [0]:
(
  spark.readStream.table('pdf_raw')
  .withColumn('url', F.col('path'))
  .withColumn('path', F.regexp_replace(F.col("path"), 'dbfs:', ''))
  .withColumn("extracted_content", F.explode(extract_pages_content("content")))
  .select(
    "path",
    "url",
    "content",
    F.col("extracted_content.title").alias("title"),
    F.col("extracted_content.page_number").alias("page_number"),
    F.col("extracted_content.text").alias("text")
  )
  .withColumn("content", F.explode(split_in_chunks("text")))
  .withColumn("embedding", get_embedding("content"))
  # .withColumn('url', F.concat(F.lit(BASE_URL), F.element_at(F.split(F.col('path'), '/'), -1)))
  .select('path', 'title', 'url', F.col('page_number').cast('int'), 'content', 'embedding')
  .writeStream
    .trigger(availableNow=True)
    .option("checkpointLocation", f'dbfs:{volume_folder}/checkpoints/pdf_chunk')
    # .table(PDFS_TABLE_FULLNAME.split('.')[-1])
    .table("hackathon_pdfs_test")
    .awaitTermination()
)

#Let's also add our documentation web page from the simple demo (make sure you run the quickstart demo first)
# if table_exists(PDFS_TABLE_FULLNAME):
#   (spark.readStream
#    .option("skipChangeCommits", "true") # ignore transactions that delete or modify existing records
#    .table('pdf_raw') #skip changes for more stable demo
#   .withColumn('embedding', get_embedding("content"))
#   .select('url', 'content', 'embedding')
#   .writeStream
#     .trigger(availableNow=True)
#     .option("checkpointLocation", f'dbfs:{volume_folder}/checkpoints/docs_chunks')
#     .table(PDFS_TABLE_FULLNAME.split('.')[-1]).awaitTermination())

In [0]:
display(
    spark.sql(f"""
        SELECT * FROM hackathon_pdfs_test limit 10
    """)
)

id path title url page_number content embedding 1 /Volumes/test-catalog/bronze/volume_hackathon/pdfs/HKN_trailer_axles_Spare_parts_list_BPW_31022101e_2021.pdf BPW Original spare parts / BPW trailer axles series H.. / R../ K.. / N.. with drum brakes / EL-HKN 31022101e dbfs:/Volumes/test-catalog/bronze/volume_hackathon/pdfs/HKN_trailer_axles_Spare_parts_list_BPW_31022101e_2021.pdf 0 BPW Original spare parts
BPW trailer axles series H.. / R../ K.. / N.. with 
drum brakesH
RK
N
BPW-EL-HKN 31022101e List(0.09448242, -0.5991211, 1.5, 0.42211914, -1.2333984, 0.55126953, -1.3037109, -0.15576172, -0.64990234, 0.10546875, 0.0904541, -0.12005615, -0.5415039, 0.19812012, 0.40039062, -0.7060547, -0.36889648, -0.91845703, -0.70996094, 0.42333984, 0.8515625, -0.76660156, 0.018844604, -0.9277344, 0.45092773, 0.60839844, -0.77441406, 0.83496094, 0.78125, -0.5527344, -0.06567383, 0.8276367, -1.6103516, 0.8046875, -0.82666016, -0.48632812, -0.7421875, 1.296875, 0.22668457, -0.5263672, 0.765625, 0.48657227, -0.13208008, -0.16137695, 0.9555664, 0.8833008, -0.20568848, 0.03475952, -1.1455078, 0.008430481, 0.09136963, -0.2614746, 0.5292969, 0.020431519, 0.07366943, 0.27514648, -0.022705078, 0.68847656, 0.5957031, 0.44848633, -0.022567749, 0.40820312, -0.057861328, -1.1064453, -0.12322998, 0.043914795, -0.7163086, 1.5664062, -0.8120117, 0.6225586, -0.5449219, -0.18603516, 0.034576416, 0.3659668, 1.203125, -0.66259766, -0.08654785, 0.3317871, -0.9013672, -1.59375, -0.46679688, 0.4560547, -1.2490234, -0.9692383, -0.38256836, -1.625, -0.20092773, -0.078063965, 0.09069824, 0.2142334, 0.33764648, -0.71972656, -2.4485588E-4, -0.08013916, -0.66503906, -0.10961914, -0.49072266, -0.8725586, -0.20349121, 0.6088867, 1.0146484, 0.1809082, 0.7006836, -1.0332031, -1.4814453, 0.6899414, -0.81591797, -0.45288086, 1.0361328, 0.33740234, 0.21972656, 0.33276367, -0.007698059, -0.053649902, -0.14294434, 0.40893555, -0.16662598, -0.55615234, -0.20959473, 0.30126953, 0.5698242, 0.006866455, 0.7817383, 0.14123535, 1.5605469, 1.109375, -0.34423828, -0.32983398, -0.064208984, 1.0488281, -0.18273926, -0.58447266, 0.17810059, 0.38452148, -0.8276367, -0.82958984, -0.6381836, -0.37963867, 0.44433594, -0.88671875, -1.078125, 0.2746582, 0.17224121, 0.30444336, 0.36083984, 0.3881836, -0.50439453, 0.30981445, 0.060150146, 0.06341553, 0.0149002075, -0.057495117, -0.37329102, -0.2626953, 0.6074219, -1.0185547, 0.578125, 0.38183594, 0.7807617, -0.17175293, -0.24645996, -0.007320404, -0.21276855, 2.0527344, 0.16137695, 1.6445312, -0.33789062, -0.12573242, 0.32592773, -0.5336914, 0.39282227, 1.1103516, 0.22460938, -0.55078125, -0.26635742, -0.07342529, 0.4025879, 0.20385742, -0.24023438, -1.2226562, -0.2890625, -0.20788574, 0.546875, -0.21691895, 0.86572266, -0.55371094, 0.0127334595, 0.25024414, -0.0690918, -0.43139648, 0.2902832, -1.5361328, -0.6748047, 0.24780273, -1.5625, 0.1348877, -0.14501953, -0.60009766, -0.1459961, -0.73779297, -0.7841797, -0.0692749, -0.78125, -0.99902344, 0.03378296, 0.4423828, 0.7988281, -0.13537598, 0.62402344, -0.49560547, 0.3083496, -0.05517578, -0.025466919, -1.5449219, -0.023880005, -0.4404297, 0.6088867, -1.1533203, 0.014587402, 0.7573242, -0.3786621, -0.47021484, -0.86035156, -1.9296875, 0.7270508, -0.48095703, 0.42871094, -0.7416992, 0.7470703, -0.41748047, 0.32055664, 1.2041016, -1.4794922, -1.4169922, 0.72558594, -0.71972656, -0.27001953, 1.6123047, -0.6479492, -0.9433594, 0.9169922, -0.062683105, 0.3815918, 0.24523926, -0.15185547, -0.43676758, 0.47460938, 0.0357666, -0.87109375, 0.017181396, -0.36376953, -0.13171387, 0.375, 0.4362793, -0.83935547, -0.9145508, -0.13928223, 0.3852539, 0.0032482147, -0.047973633, -0.28637695, 0.57373047, 0.73095703, 0.2322998, 0.23620605, 0.26586914, -0.52685547, 0.20568848, -0.17260742, 0.08026123, 0.7246094, 0.36938477, -1.1152344, -0.50097656, 0.6303711, 0.3876953, 0.31079102, -1.0791016, -0.032836914, 0.14050293, 0.2298584, 0.72802734, 0.33496094, 0.23547363, 0.40576172, -0.73535156, -0.48779297, 0.3317871, 0.129


### Our dataset is now ready! Let's create our Self-Managed Vector Search Index.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-pdf-self-managed-3.png?raw=true" style="float: right; width: 600px; margin-left: 10px">

Our dataset is now ready. We chunked the documentation pages into small sections, computed the embeddings and saved it as a Delta Lake table.

Next, we'll configure Databricks Vector Search to ingest data from this table.

Vector search index uses a Vector search endpoint to serve the embeddings (you can think about it as your Vector Search API endpoint). <br/>
Multiple Indexes can use the same endpoint. Let's start by creating one.

In [0]:
if not endpoint_exists(VSC, VECTOR_SEARCH_ENDPOINT_NAME):
    VSC.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(VSC, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

# vsc.list_endpoints()
# vsc.list_indexes(VECTOR_SEARCH_ENDPOINT_NAME)

INFO:py4j.clientserver:Received command c on object id p0


Waiting for endpoint to be ready, this can take a few min... {'name': 'rag_endpoint', 'creator': "stefano.d'angelo@tip-group.com", 'creation_timestamp': 1728324018563, 'last_updated_timestamp': 1728324018563, 'endpoint_type': 'STANDARD', 'last_updated_user': "stefano.d'angelo@tip-group.com", 'id': '2eefae09-7845-4bdd-ac59-c120f7cad98e', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'rag_endpoint', 'creator': "stefano.d'angelo@tip-group.com", 'creation_timestamp': 1728324018563, 'last_updated_timestamp': 1728324018563, 'endpoint_type': 'STANDARD', 'last_updated_user': "stefano.d'angelo@tip-group.com", 'id': '2eefae09-7845-4bdd-ac59-c120f7cad98e', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'rag_endpoint', 'creator': "stefano.d'angelo@tip-group.com", 'creation_timestamp': 1728324018563, 'last_updated_timestamp': 


You can view your endpoint on the [Vector Search Endpoints UI](#/setting/clusters/vector-search). Click on the endpoint name to see all indexes that are served by the endpoint.

In [0]:
# from databricks.sdk import WorkspaceClient
# import databricks.sdk.service.catalog as c

if not index_exists(VSC, VECTOR_SEARCH_ENDPOINT_NAME, VS_INDEX_FULLNAME.replace('`', '')):
  print(f"Creating index {VS_INDEX_FULLNAME} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  try:
    VSC.create_delta_sync_index(
      endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
      index_name=VS_INDEX_FULLNAME.replace('`', ''),
      # source_table_name=PDFS_TABLE_FULLNAME.replace('`', ''), #The table we'd like to index
      source_table_name=TABLE_PATH.format(table_name="hackathon_pdfs_test").replace('`', ''),
      pipeline_type="TRIGGERED", #Sync needs to be manually triggered
      primary_key="id",
      embedding_dimension=1024, #Match your model embedding size (gte)
      embedding_vector_column="embedding"
    )
  except Exception as e:
    if "already exists" in str(e):
      pass
    else:
      raise e
  #Let's wait for the index to be ready and all our embeddings to be created and indexed
  wait_for_index_to_be_ready(VSC, VECTOR_SEARCH_ENDPOINT_NAME, VS_INDEX_FULLNAME.replace('`', ''))
else:
  #Trigger a sync to update our vs content with the new data saved in the table
  wait_for_index_to_be_ready(VSC, VECTOR_SEARCH_ENDPOINT_NAME, VS_INDEX_FULLNAME.replace('`', ''))
  try:
    VSC.get_index(VECTOR_SEARCH_ENDPOINT_NAME, VS_INDEX_FULLNAME.replace('`', '')).sync()
  except Exception as e:
    print(e)

INFO:py4j.clientserver:Received command c on object id p0


Creating index `test-catalog`.bronze.hackathon_pdfs_self_managed_vs_index on endpoint rag_endpoint...
Waiting for index to be ready, this can take a few min... {'detailed_state': 'PROVISIONING_INDEX', 'message': 'Delta sync Index creation is pending. Check latest status: https://adb-4176489020682060.0.azuredatabricks.net/explore/data/test-catalog/bronze/hackathon_pdfs_self_managed_vs_index', 'indexed_row_count': 0, 'ready': False, 'index_url': 'adb-4176489020682060.0.azuredatabricks.net/api/2.0/vector-search/endpoints/rag_endpoint/indexes/test-catalog.bronze.hackathon_pdfs_self_managed_vs_index'} - pipeline url:adb-4176489020682060.0.azuredatabricks.net/api/2.0/vector-search/endpoints/rag_endpoint/indexes/test-catalog.bronze.hackathon_pdfs_self_managed_vs_index


## Searching for similar content

That's all we have to do. Databricks will automatically capture and synchronize new entries in your Delta Lake Table.

Note that depending on your dataset size and model size, index creation can take a few seconds to start and index your embeddings.

Let's give it a try and search for similar content.

*Note: `similarity_search` also supports a filters parameter. This is useful to add a security layer to your RAG system: you can filter out some sensitive content based on who is doing the call (for example filter on a specific department based on the user preference).*

In [0]:
from mlflow.deployments import get_deploy_client
deploy_client = get_deploy_client("databricks")

question = "How can I replace a disc brake?"

response = deploy_client.predict(endpoint="databricks-gte-large-en", inputs={"input": [question]})
embeddings = [e['embedding'] for e in response.data]

results = VSC.get_index(VECTOR_SEARCH_ENDPOINT_NAME, VS_INDEX_FULLNAME.replace('`', '')).similarity_search(
  query_vector=embeddings[0],
  columns=["url", "content", "page_number"],
  num_results=3)
docs = results.get('result', {}).get('data_array', [])
print(docs[0])
# pprint(docs)

['https://www.bpw.de/fileadmin/user_upload/Service/Downloads/Trailer_Disc_brake_TSB-ECODisc-_Workshop_manual_2023_en-BPW-35292301e.pdf', 'Replacing the brake disc 14\n\n ���\n\n_Picture 60_', 95.0, 0.004604547]


In [0]:
results

{'manifest': {'column_count': 4,
  'columns': [{'name': 'url'},
   {'name': 'content'},
   {'name': 'page_number'},
   {'name': 'score'}]},
 'result': {'row_count': 3,
  'data_array': [['https://www.bpw.de/fileadmin/user_upload/Service/Downloads/Trailer_Disc_brake_TSB-ECODisc-_Workshop_manual_2023_en-BPW-35292301e.pdf',
    'Replacing the brake disc 14\n\n ���\n\n_Picture 60_',
    95.0,
    0.004604547],
   ['https://www.bpw.de/fileadmin/user_upload/Service/Downloads/Trailer_Disc_brake_TSB-ECODisc-_Workshop_manual_2023_en-BPW-35292301e.pdf',
    '14 Replacing the brake disc\n\n 14.5 Replacing the brake disc for axles with ECO Plus Unit:\n - TSB 3709 (10 t)\n - TSB 4312\n\n[1] Prevent the vehicle from moving away.\nRelease the service and parking brakes.\n\n[2] Unscrew the hubcap (460) from the wheel hub.\n\n[3] Loosen wheel nuts.\n\n[4] Support vehicle safely.\n\n[5] Raise axle until the tyres are free.\n\n[6] Unscrew wheel nuts and remove the wheel from\nthe hub.\n\n[7] Remove the se